### Roberta classifer

In [2]:
import datasets
import pandas as pd
from datasets import Dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import torch

In [ ]:
def make_submission():
    roberta_model = RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta-trained-0-1/finetuned_model/").to("cuda")
    tokenizer = RobertaTokenizerFast.from_pretrained("/kaggle/input/roberta-trained-0-1/finetuned_model/")
    
    test_df = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
    test_dataset = Dataset.from_pandas(test_df)
    
    submisson_dataset = test_dataset.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True), batched=True)
    # final test dataset
    test_dataset = Dataset.from_pandas(pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv"))
    
    def predict(batch):
        batch = {k:torch.tensor(v).to("cuda") for k,v in batch.items() if k in ["input_ids","attention_mask"]}
        with torch.no_grad():
            output = roberta_model(**batch)
        logits = output.logits.detach().cpu()
        probs_generated = logits.softmax(dim=1)[:,1]
        return {'generated': probs_generated.tolist()}
    
    submisson_results = submisson_dataset.map(lambda x: predict(x), batched=True)
    submisson_results.to_pandas()[["id", "generated"]].to_csv("submission.csv", index=False)

In [ ]:
make_submission()